# IPFS Concepts - SCRATCHPAD

## IPFS Service Node

A node that is running the IPFS daemon and is connected to the IPFS network. It is able to provide content to other nodes and to retrieve content from other nodes.

In [1]:
# docker compose -f "tests/ipfs/docker-compose.yml" up -d --build

### Default Server Address

In [2]:
__IPFS_DEFAULT_URL__ = '/ip4/127.0.0.1/tcp/5001'

## IPFS-Toolkit

In [3]:
# pip install IPFS-Toolkit

### Connect

In [4]:
import ipfs_api

# Connect to the IPFS daemon
def connect():
    client = ipfs_api.ipfshttpclient
    return client.connect(__IPFS_DEFAULT_URL__)

#### Check Connection

In [5]:
def print_status():
    ipfs = connect()
    print('Client ID: ', ipfs.id())
    print('Client Version: ', ipfs.version())

print_status()

Client ID:  <ipfshttpclient2.client.base.ResponseBase: {'ID': '12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', 'PublicKey': 'CAESILveCqhrvYi7xfZkCFoytPVThX++ztHAh85jBT3xQ0Et', 'Addresses': ['/ip4/108.61.182.184/tcp/4001/p2p/12D3KooWPuZGkjnjLso3sLgNm5qaQMgiX77ndQ14JHqR2TwWp3Be/p2p-circuit/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/108.61.182.184/udp/4001/quic-v1/p2p/12D3KooWPuZGkjnjLso3sLgNm5qaQMgiX77ndQ14JHqR2TwWp3Be/p2p-circuit/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/108.61.182.184/udp/4001/quic/p2p/12D3KooWPuZGkjnjLso3sLgNm5qaQMgiX77ndQ14JHqR2TwWp3Be/p2p-circuit/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/127.0.0.1/tcp/4001/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/127.0.0.1/udp/4001/quic-v1/p2p/12D3KooWNTinNPaFXPyoo2wGcrjFC4uekMgDM5pFY46zwxxgL8HA', '/ip4/127.0.0.1/udp/4001/quic-v1/webtransport/certhash/uEiAsG_OZi39bk9tp4XO6luYhWxo_vmHB30yD1HFNsCV1hg/certhash/uEiBU9zqLAAbnHBe6T6br9Bg29WL

### Publish

#### Strings

In [6]:
def add_str(string: str):
    with connect() as c:
        return c.add_str(string)

returned_hash = add_str(u'Hello World!')
print('CIDV0: ', returned_hash)

# convert to CIDv1
returned_hash = make_cid(returned_hash).to_v1()

print('CIDV1: ', returned_hash)

CIDV0:  Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy


NameError: name 'make_cid' is not defined

#### Bytes

In [ ]:
from cid import make_cid

def add_bytes(data: bytes):
    with connect() as c:
        return c.add_bytes(data)
    
returned_hash = add_bytes(b'Hello World!')
print('CIDV0: ', returned_hash)

# convert to CIDv1
returned_hash = make_cid(returned_hash).to_v1()

print('CIDV1: ', returned_hash)

CIDV0:  Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy
CIDV1:  zdj7Wn77GBmxYDXRUaaaKY6wPYZGvn1omPeUTYCf27aD2Tu9u


#### JSON

In [ ]:
from cid import CIDv0, CIDv1, make_cid

def add_json(json: dict):
    with connect() as c:
        return c.add_json(json)
    
returned_hash = add_json({'hello': 'world'})
print('CIDV0: ', returned_hash)

# convert to CIDv1
returned_hash = make_cid(returned_hash).to_v1()

print('CIDV1: ', returned_hash)

CIDV0:  QmNrEidQrAbxx3FzxNt9E6qjEDZrtvzxUVh47BXm55Zuen
CIDV1:  zdj7WVwV64Uhe5ZpkZvh7qm6UgHrRmJcUR6Lp6amyvRtX8opi


#### Files

In [ ]:
def add_file(path: str):
    with connect() as c:
        return c.add(path)
    
returned_hash = add_file('concepts_cid_test.json')
print('CIDV0: ', returned_hash['Hash'])

# convert to CIDv1
returned_hash = make_cid(returned_hash['Hash']).to_v1()

print('CIDV1: ', returned_hash)

CIDV0:  QmUJarh6GuMaNgpkvkC1CoRk8qDk6J39CopoYi9fVdSEiv
CIDV1:  zdj7WbPqE8A8NqBFQ8gfV9d5BGJm3RBoqTH58ELDWYLK518tr


#### Blocks

##### Raw Blocks

In [ ]:
import io
import base58
import json
# import BytesIO
from typing import Callable, Iterable
from cid import CIDv1, make_cid, CIDv0
# from multiformats_cid import make_cid, CIDv0, CIDv1, cid
# from Crypto.Hash import SHA256
import hashlib
from merkly.mtree import MerkleTree
import multihash
import codecs
import ipfs_api
import multibase

def as_chunks(stream: io.BytesIO, chunk_size: int) -> Iterable[bytes]:
    while len(chunk := stream.read(chunk_size)) > 0:
        yield chunk

def chunk_to_leaf(chunk: bytes) -> str:
    return hashlib.sha256(chunk).hexdigest()

def data_to_tree(data: bytes, chunk_size: int) -> list[str]:
    # Create a buffer to hold the tree
    tree: list[str] = []

    # Iterate over the chunks
    for chunk in as_chunks(io.BytesIO(data), chunk_size):
        # Calculate the leaf hash
        tree.append(chunk_to_leaf(chunk))
        # print(tree)

    # Return the tree
    return tree

def tree_to_root(tree: list[bytes]) -> str:
    sha256_hash_funciton: Callable[[str], str] = lambda x, y: str(hashlib.sha256(x.encode() + y.encode()).hexdigest())

    # Calculate the root hash
    mt = MerkleTree(tree, sha256_hash_funciton)
    root = mt.root
    print(f'raw_root: {root}')
    # root = codecs.decode(root, 'hex')
    # print('root:', root.hex())
    return root

# Connect to the local IPFS node
client = ipfs_api.ipfshttpclient.connect('/ip4/127.0.0.1/tcp/5001')

# Read the file
with open('concepts_cid_test.json', 'rb') as f:
    data = f.read()

# Calculate the tree
tree = data_to_tree(data, 4)

# Calculate the root
root = tree_to_root(tree)

# Add the Merkle root to IPFS
res = client.block.put(file='concepts_cid_test.json')

print(res)  # prints the CID of the Merkle root

# Get the block from IPFS
block = client.block.get(res['Key'])

# Print the block
print(block.decode())


raw_root: fe9b05ddd2c30b4134f8b5e1b9ecf8b9f8485b8090565d006ec009b9c752f308
<ipfshttpclient2.client.base.ResponseBase: {'Key': 'bafkreicpuquhapdqadlj6uv2hhx6vma4q2fhi7hkh624swguwj3zw4edwi', 'Size': 338}>
{
  "name": "John Doe",
  "age": 30,
  "email": "johndoe@example.com",
  "address": {
    "street": "123 Main St",
    "city": "Anytown",
    "state": "CA",
    "zip": "12345"
  },
  "phoneNumbers": [
    {
      "type": "home",
      "number": "555-555-1234"
    },
    {
      "type": "work",
      "number": "555-555-5678"
    }
  ]
}



##### Creating a Block - Get The CID

In [ ]:
import hashlib
import multihash
import base58
import cbor2
from io import BytesIO
import cid

# Your node
node = {
    'data': 'Hello World!',
    'links': [],
}

# Serialize the node using DAG-CBOR
serialized = cbor2.dumps(node)

# Calculate the SHA-256 hash of the serialized node
hash = hashlib.sha256(serialized).digest()

# Create a multihash from the hash
mh = multihash.encode(hash, 'sha2-256')

# Create a CIDv1 from the multihash
cid_ = base58.b58encode(mh).decode()
print(cid_)


Qmb9cDaiMqrbBoYoL8MvZLjLwxgW6RCSwURNRxgNKcWPo8


In [ ]:

print(cid_)  # prints the CID of the node

# Your Merkle DAG
dag = {
    'data': 'Hello World!',
    'links': [
        # {'cid': 'bafyreihdwdcefihqacbuea4s2g3ktn7nol4n6ie3jprsg2jvh2hjcxzrna', 'name': 'child1', 'size': 13},
        # {'cid': 'bafyreif447ug2lstf5abwxzzjnsrh3flmhfcdy2s2n5q4ckq57qj2uv2sm', 'name': 'child2', 'size': 13},
    ],
}

# Serialize the DAG node using DAG-CBOR
serialized = cbor2.dumps(dag)

# Add the serialized DAG node to IPFS
res = client.block.put(BytesIO(serialized))
print(res)  # prints the CID of the Merkle root

# Get the block from IPFS
block = client.block.get(res['Key'])
print(block)
print(cid.make_cid(res['Key']))

cidv0 = cid.make_cid(cid_)
print(cidv0)
cidv1 = cidv0.to_v1()
print(cidv1)

res1 = client.add_bytes(b'Hello World!')
print(res1)

res2 = client.add(BytesIO(serialized))
print(res2)

print(client.cat(res2['Hash']))
print(client.cat(res1))

Qmb9cDaiMqrbBoYoL8MvZLjLwxgW6RCSwURNRxgNKcWPo8
<ipfshttpclient2.client.base.ResponseBase: {'Key': 'bafkreif6ko4ucr2qtihpnqlfqatklokmsczhsvv6ycvebdilcxxrlzdyim', 'Size': 26}>
b'\xa2ddatalHello World!elinks\x80'
zb2rhjTEhdvbh7c8zQMKE8DFF7oiEPzouXBrFDSjdZx5tUZzr
Qmb9cDaiMqrbBoYoL8MvZLjLwxgW6RCSwURNRxgNKcWPo8
zdj7WiErb1nDKLC4Wrj4sKYRiZuaAswoxcaonpu6m53945Hy4
Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy
<ipfshttpclient2.client.base.ResponseBase: {'Name': 'QmYrWm8tZNwUMTbuDvB7vbXUQKLtrhGBJWjMxCHfjUbWTs', 'Hash': 'QmYrWm8tZNwUMTbuDvB7vbXUQKLtrhGBJWjMxCHfjUbWTs', 'Size': '34'}>
b'\xa2ddatalHello World!elinks\x80'
b'Hello World!'


In [ ]:
import hashlib
import multihash
import base58
import cbor2
from cid import make_cid

# Your data
data = b'Hello World!'

dag = {
    'data': data,
    'links': [],
}

# Serialize the data using DAG-CBOR
serialized = cbor2.dumps(dag)

# Calculate the SHA-256 hash of the serialized data
hash = hashlib.sha256(serialized).digest()

# Create a multihash from the hash
mh = multihash.encode(hash, 'sha2-256')

# Create a CIDv1 from the multihash
# 0x71 is the multicodec for DAG-CBOR
cid_ = make_cid(1, 'dag-cbor', mh)

print(cid_)  # prints the CID of the data

zdpuAwniF6gSgt1pgdFydH3LPG5o4KcTzkRTY76dmnyHUTdAF


In [ ]:
import hashlib

# Node structure
class Node:
    def __init__(self, data):
        self.data = data
        self.children = []

    def hash(self):
        m = hashlib.sha256()
        m.update(self.data.encode('utf-8'))
        for child in self.children:
            m.update(child.hash().encode('utf-8'))
        return m.digest()
    
def hash_to_cid(hash: str):
    mh = multihash.encode(hash, 'sha2-256')
    cid_ = make_cid(1, 'dag-pb', mh)
    return cid_
# Create nodes
node1 = Node('Hello, world!')
# node2 = Node('node2')
# node3 = Node('node3')
# node4 = Node('node4')

# Link nodes to create a DAG
# node1.children.append(node2)
# node1.children.append(node3)
# node2.children.append(node4)
# node3.children.append(node4)

# Hash the DAG starting from the root
print(node1.hash().hex())
print(hash_to_cid(node1.hash()))
print(client.add_str(node1.data))

315f5bdb76d078c43b8ac0064e4a0164612b1fce77c869345bfc94c75894edd3
zdj7WYkdPhBE4a3b6sm9hKakvrDguPBFZiQcqLSdisNZxKznW
QmWGeRAEgtsHW3ec7U4qW2CyVy7eA2mFRVbk1nb24jFyks


In [ ]:
def chunk_data(data, chunk_size):
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

# Your data
data = 'Hello World!'

# Chunk the data into pieces of size 256 kilobytes
chunks = chunk_data(data, 256 * 1024)

for chunk in chunks:
    print(chunk)
    print(hashlib.sha256(chunk.encode()).hexdigest())
    print(hash_to_cid(hashlib.sha256(chunk.encode()).digest()))

with open('concepts_cid_test.json', 'rb') as f:
    data = f.read()

# Chunk the data into pieces of size 256 kilobytes
chunks = chunk_data(data, 256 * 1024)

for chunk in chunks:
    print(chunk)
    print(hashlib.sha256(chunk).hexdigest())
    print(hash_to_cid(hashlib.sha256(chunk).digest()))

Hello World!
7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069
zdj7We1fKtuSjWA1zhER12jc7mydHG43F22AyC8eLrBaBF9Da
b'{\n  "name": "John Doe",\n  "age": 30,\n  "email": "johndoe@example.com",\n  "address": {\n    "street": "123 Main St",\n    "city": "Anytown",\n    "state": "CA",\n    "zip": "12345"\n  },\n  "phoneNumbers": [\n    {\n      "type": "home",\n      "number": "555-555-1234"\n    },\n    {\n      "type": "work",\n      "number": "555-555-5678"\n    }\n  ]\n}\n'
4fa428703c7000d69f52ba39efeab01c868a747cea3fb5c958d4b2779b7083b2
zdj7WannUB2kdHy7pr3iNhw9sxs7b1EZphdbnsiJkZfGZi8o3


In [ ]:
from multiformats import CID, multibase, multicodec
import multihash

# Create a CIDv0
cidv0 = CID(
    base='base58btc',
    version=0,
    codec='dag-pb',
    digest=multihash.encode(hashlib.sha256(b'Hello World!').digest(), 'sha2-256'),
)

print(cidv0.human_readable)
print(cidv0)

# Create a CIDv1
cidv1 = CID(
    base='base58btc',
    version=1,
    codec='dag-pb',
    digest=multihash.encode(hashlib.sha256(b'Hello World!').digest(), 'sha2-256', 32),
)

print(cidv1.human_readable)

mh = multihash.encode(hashlib.sha256(b'Hello World!').digest(), 'sha2-256')
print(mh.hex())

# mc_c = multicodec.wrap(multicodec.get(code=0x01), mh)
# print(mc_c)

# mc_2 = multicodec.wrap('dag-pb', mc_c)
# print(mc_2)



mb = multibase.encode(mh, 'base58btc')
print(mb[1::])

# res = client.cat(mb[1:])
# print(res)



base58btc - cidv0 - dag-pb - (sha2-256 : 256 : 7F83B1657FF1FC53B92DC18148A1D65DFC2D4B1FA3D677284ADDD200126D9069)
QmWvQxTqbG2Z9HPJgG57jjwR154cKhbtJenbyYTWkjgF3e
base58btc - cidv1 - dag-pb - (sha2-256 : 256 : 7F83B1657FF1FC53B92DC18148A1D65DFC2D4B1FA3D677284ADDD200126D9069)
12207f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069
QmWvQxTqbG2Z9HPJgG57jjwR154cKhbtJenbyYTWkjgF3e


In [ ]:

# print(multibase.encode(base58.b58encode(multihash.encode(hashlib.sha256(b'Hello World!').digest(), 'sha2-256', 32), 'base58btc').decode()))
print(cidv1)

# get the cidv0
cidv1_digest = cidv1.digest
print(cidv1_digest.hex())

created = cid.make_cid(str(cidv1))
print(created)

# get the cidv1
s = str(created.to_v0())
print(s)
try:
    print(client.cat(s))
except:
    print('not found')

# Create a CIDv1
res = client.add_str('Hello World!')
print(res)

find = client.cat(res)
print(find)

zdj7We1fKtuSjWA1zhER12jc7mydHG43F22AyC8eLrBaBF9Da
12207f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069
zdj7We1fKtuSjWA1zhER12jc7mydHG43F22AyC8eLrBaBF9Da
QmWvQxTqbG2Z9HPJgG57jjwR154cKhbtJenbyYTWkjgF3e
not found
Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy
b'Hello World!'


In [ ]:
from bases import make


class Node:
    def __init__(self, data):
        self.data = data
        self.links = []

    def serialize_oopt1(self):
        return {
            'data': self.data.decode('utf-8'),
            'links': [child.serialize_oopt1() for child in self.links],
        }
    
    def serialize_(self):
        return json.dumps(self.serialize_oopt1(), default=lambda o: o.__dict__, sort_keys=True, indent=4)

def chunk_data(data, chunk_size):
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

def create_dag(data, chunk_size):
    chunks = chunk_data(data, chunk_size)
    nodes = [Node(chunk) for chunk in chunks]
    for i in range(len(nodes) - 1):
        nodes[i].children.append(nodes[i+1])
    return nodes[0]  # Return the root of the DAG

# Your binary data
data = b'Hello World!'

# Create a DAG with chunks of size 5
root = create_dag(data, 256 * 1024)
print(root)

# serialized = root.serialize_oopt1()
# print(serialized)

# mh = multihash.encode(hashlib.sha256(serialized['data'].encode()), 'sha2-256')
# print(mh)

serialized_2 = root.serialize_()
print(serialized_2)

mh_2 = multihash.encode(hashlib.sha256(serialized_2.encode()).digest(), 'sha2-256')
print(mh_2)

cidv1_ = CID(
    base='base58btc',
    version=1,
    codec='dag-pb',
    digest=mh_2,
)

print(cidv1_)

# added = client.block.put(io.BytesIO(serialized))
# print(added)

make_cid_ = make_cid(str(cidv1_))
print(make_cid_)

added = client.block.put(io.BytesIO(serialized_2.encode()))
print(added)

fetched = client.block.get(added['Key'])
print(fetched)




{
    "data": "Hello World!",
    "links": []
}
b'\x12 \x07yZ\x8b\xf2\x12<\xb3\x84\t\xfcS\x80\xbe\x82\x0e\x12\xa7hct[\xf3um\x07\xed\x06\xb5\xa9f\xc0'
zdj7WVw5FxkQoTca7bJ1rwri5c75oL45HswjBqD3jn2NZtARu
zdj7WVw5FxkQoTca7bJ1rwri5c75oL45HswjBqD3jn2NZtARu
<ipfshttpclient2.client.base.ResponseBase: {'Key': 'bafkreiahpfnix4qshszyicp4koal5aqocktwqy3ulpzxk3ih5udllklgya', 'Size': 47}>
b'{\n    "data": "Hello World!",\n    "links": []\n}'


In [ ]:
import ipfs_api
import ipfs_datatransmission
import ipfs_cli

# Connect to the local IPFS node
client = ipfs_api.ipfshttpclient.connect(__IPFS_DEFAULT_URL__)

# Read the file
with open('concepts_cid_test.json', 'rb') as f:
    data = f.read()

# Calculate the tree
prediction = ipfs_api.predict_cid('concepts_cid_test.json')
print(prediction)

predidction_2 = ipfs_api.predict_cid(io.BytesIO(b'Hello World!'))
print(predidction_2)

print(ipfs_api.pins())

# # Calculate the root
# root = ipfs_datatransmission.tree_to_root(tree)

# Add the Merkle root to IPFS
res = client.block.put(file='concepts_cid_test.json')

print(res)  # prints the CID of the Merkle root


QmUJarh6GuMaNgpkvkC1CoRk8qDk6J39CopoYi9fVdSEiv
Qmf1rtki74jvYmGeqaaV51hzeiaa6DyWc98fzDiuPatzyy
{'QmNaPyRfA9pKxfoZ2obQh3nPF783dpHSRCHBECghKiyJky': {'Type': 'recursive'}, 'QmNrEidQrAbxx3FzxNt9E6qjEDZrtvzxUVh47BXm55Zuen': {'Type': 'recursive'}, 'QmPG7z9cvzPajnEEHqvEasLscjYmUXDWw9askfW5VNWY1T': {'Type': 'recursive'}, 'QmRf22bZar3WKmojipms22PkXH1MZGmvsqzQtuSvQE3uhm': {'Type': 'recursive'}, 'QmSg65W6sgWFDPDesEGwuMAtGPBZznaAB7fzJF914khbUW': {'Type': 'recursive'}, 'QmT4vBkDXGTiALx8Z2TEdsiyvL9cWU2AZeexjKoSnPtLX3': {'Type': 'recursive'}, 'QmTPXo5EBG99iEKxgL92UjcYKmkW7zUGy65b5b1mCFWS93': {'Type': 'recursive'}, 'QmTVfsj9xHGa6vZsoZLWoZRkKWdWjfEMBQahS7tXn4foqS': {'Type': 'recursive'}, 'QmUJarh6GuMaNgpkvkC1CoRk8qDk6J39CopoYi9fVdSEiv': {'Type': 'recursive'}, 'QmUNLLsPACCz1vLxQVkXqqLX5R1X345qqfHbsf67hvA3Nn': {'Type': 'recursive'}, 'QmUwNnn1CUqWFWYFFwWGLTywLrVu3V33kBXLyrXVG56mh4': {'Type': 'recursive'}, 'QmVSGXxECDigYrXAdc5kg6wgunts4rMdq7CdsGkt76wLFp': {'Type': 'recursive'}, 'QmVrf8MXjvt8LWs3breFwKvsCa2L

In [ ]:
from cid import CIDv1
from multihash import multihash
from multicodec import add_prefix
import hashlib

# Your data
data = b'Hello World!'

# Serialize the data (in this case, we just use the raw data)
serialized_data = data

# Hash the data
hashed_data = hashlib.sha256(serialized_data).digest()

# Create the multihash
mh = multihash.encode(hashed_data, 'sha2-256')

# Create the CID
cid = CIDv1('dag-pb', mh)

# Print the CID
print(cid)



zdj7We1fKtuSjWA1zhER12jc7mydHG43F22AyC8eLrBaBF9Da
